In [ ]:
import pandas as pd

artigo_treino = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv")

In [ ]:
!python -m spacy download pt_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 55.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [ ]:
texto = "Adoro a cidade de Niterói"
doc = nlp(texto)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
doc.ents

(Adoro, Niterói)

In [ ]:
doc[0].is_stop

False

In [ ]:
doc[1].is_alpha

True

In [ ]:
def trata_textos(doc):
  token_validos = []

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      token_validos.append(token.text)

  if len(token_validos) > 2:
    return " ".join(token_validos)

In [ ]:
texto = "Adoro a cidade 212 121212121313131 3131313 %@#@@$%@$!$! $#!$ de Niterói"
doc = nlp(texto)
trata_textos(doc)

'Adoro cidade Niterói'

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size=1000, n_process=-1)]

In [ ]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [46]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

84466
84466


In [47]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [49]:
w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [50]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 

In [51]:
w2v_modelo.corpus_count

84466

In [56]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import word2vec

class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0
    self.loss_previous_step = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
      print("Loss após primeira época {}: {}".format(self.epoch, loss))
    else:
      print("Loss após época {}: {}".format(self.epoch, loss - self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [57]:
w2v_modelo.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 compute_loss=True,
                 callbacks=[callback()])

Loss após primeira época 0: 126331.8984375
Loss após época 1: 115576.2578125
Loss após época 2: 117315.96875
Loss após época 3: 106742.75
Loss após época 4: 103147.1875
Loss após época 5: 99560.125
Loss após época 6: 96969.0
Loss após época 7: 104224.5625
Loss após época 8: 96786.3125
Loss após época 9: 89164.8125
Loss após época 10: 84945.375
Loss após época 11: 78952.5
Loss após época 12: 86207.75
Loss após época 13: 79946.875
Loss após época 14: 73757.625
Loss após época 15: 71995.0
Loss após época 16: 79468.625
Loss após época 17: 69878.25
Loss após época 18: 68974.875
Loss após época 19: 75351.125
Loss após época 20: 67359.0
Loss após época 21: 73390.375
Loss após época 22: 68462.625
Loss após época 23: 70308.125
Loss após época 24: 70332.5
Loss após época 25: 58569.0
Loss após época 26: 64609.75
Loss após época 27: 53072.75
Loss após época 28: 59524.5
Loss após época 29: 62183.75


(14584183, 16207260)

In [58]:
w2v_modelo.wv.most_similar("google")

[('facebook', 0.33712875843048096),
 ('amazon', 0.3235277533531189),
 ('canais', 0.3150807023048401),
 ('news', 0.312888503074646),
 ('disney', 0.3012644946575165),
 ('apple', 0.29917216300964355),
 ('tecnologia', 0.2823718190193176),
 ('youtube', 0.2744963467121124),
 ('snapchat', 0.27392685413360596),
 ('tenda', 0.26426446437835693)]

# Skipgram

In [60]:
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30,
                 compute_loss=True,
                 callbacks=[callback()])

Loss após primeira época 0: 2108548.0
Loss após época 1: 1441183.0
Loss após época 2: 1217014.5
Loss após época 3: 1082055.0
Loss após época 4: 1041312.0
Loss após época 5: 1006402.5
Loss após época 6: 917297.0
Loss após época 7: 878763.0
Loss após época 8: 806558.0
Loss após época 9: 785306.0
Loss após época 10: 766630.0
Loss após época 11: 703340.0
Loss após época 12: 722466.0
Loss após época 13: 704802.0
Loss após época 14: 687407.0
Loss após época 15: 673483.0
Loss após época 16: 655928.0
Loss após época 17: 628365.0
Loss após época 18: 503288.0
Loss após época 19: 440376.0
Loss após época 20: 508414.0
Loss após época 21: 473062.0
Loss após época 22: 490862.0
Loss após época 23: 431796.0
Loss após época 24: 450388.0
Loss após época 25: 392956.0
Loss após época 26: 454214.0
Loss após época 27: 427924.0
Loss após época 28: 419962.0
Loss após época 29: 390450.0


(14583909, 16207260)

In [61]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.43072205781936646),
 ('android', 0.4232885539531708),
 ('apple', 0.3809482157230377),
 ('yahoo', 0.3807125389575958),
 ('facebook', 0.37604862451553345),
 ('walmart', 0.370093435049057),
 ('buffett', 0.3690028488636017),
 ('anunciantes', 0.3626345992088318),
 ('patentes', 0.36016955971717834),
 ('toshiba', 0.3589443564414978)]

# Exportando

In [63]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_cbow.text", binary= False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_sg.text", binary= False)

# Criando o classificador com o nosso modelo

In [73]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras_numeros, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras_numeros:
    try:
      vetor_resultante += modelo.wv.get_vector(pn)
    except KeyError:
      pass
  return vetor_resultante

In [68]:
def tokenizador(texto):
  tokens_validos = []
  doc = nlp(texto)
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())
  return tokens_validos

In [71]:
def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x, y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

In [74]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo)

matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

In [75]:
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_treino_sg.shape)

(90000, 300)
(90000, 300)


In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

In [77]:
RL_cbow = classificador(w2v_modelo, matriz_vetores_treino_cbow, artigo_treino.category, matriz_vetores_teste_cbow, artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.70      0.75      6103
   cotidiano       0.62      0.79      0.69      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.86      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.78     20513
   macro avg       0.68      0.81      0.69     20513
weighted avg       0.82      0.78      0.79     20513



In [78]:
RL_sg = classificador(w2v_modelo, matriz_vetores_treino_sg, artigo_treino.category, matriz_vetores_teste_sg, artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.87      0.90      4663
   ilustrada       0.14      0.89      0.24       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513

